In [6]:
stats_path = "path/merged_segmentation_stats.csv"
counts_path = "path/merged_segmentation_counts.csv"
segmentation_path = "path/merged_segmentation.csv"
image_path="https://NGROKLINK.ngrok.io/notebooks/path/mosaic_DAPI_zX.tif"
n_pcs = 15
n_neighbors = 15
sample = "test"
min_cells_exp = 4

In [3]:

import squidpy as sq
import scanpy as sc
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import h5py
import json
import os
from os.path import join
from urllib.request import urlretrieve
from anndata import read_h5ad

from vitessce import (
    VitessceConfig,
    Component as cm,
    AnnDataWrapper,
    CoordinationType as ct,
    OmeTiffWrapper,
    MultiImageWrapper,
)


In [4]:
def change_origin(min_x_um,
                  min_y_um,
                  xcoord,
                  ycoord):
    """This function transforms coordinates to have a given origin # to do - 
    just make it matrix multiplica
    
    Parameters
    -----------
    min_x: X coordinate origin
    min_y: Y coordinate of origin
    xcoord: X coordinate 
    ycoord: Y coordinate 
    
    Returns
    -----------
    xcoord_rel_min: X coordinate with new origin
    ycoord_rel_min: Y coordinate with new origin
    """
    
    # convert to relative coords
    xcoord_rel = xcoord - min_x_um 
    ycoord_rel = ycoord - min_y_um 
    
    return [xcoord_rel, ycoord_rel]
    
def convert_to_px_coords(microns_per_pixel, 
                         xcoord_um, 
                         ycoord_um):
    
    """Molecule output from Vizgen is in um. This function converts um to px.
    
    Parameters
    -----------
    microns_per_pixel: Conversion microns to pixel space
    xcoord_um: X coordinate in um
    ycoord_um: Y coordinate in um
    
    Returns
    -----------
    xcoord_px: X coordinate in px
    ycoord_px: Y coordinate in px
    """
    
    # convert to px space from um
    xcoord_px = int(xcoord_um / microns_per_pixel)
    ycoord_px = int(ycoord_um / microns_per_pixel)
    
    return [xcoord_px, ycoord_px]

def molecules_to_px(microns_per_pixel, 
                    bbox_min_x_um,
                    bbox_min_y_um,
                    xcoord_um,
                    ycoord_um):
    
    """Vizgen outputs coordinates in um and the origin is not on the bounding box/area acquired. 
    This function transforms the origin to on the bounding box/area acquired and transforms
    the coordinates to pixel space. 
    
    Parameters
    -----------
    microns_per_pixel: Conversion microns to pixel space
    bbox_min_x_um: X coordinate of bounding box origin in um
    bbox_min_y_um: Y coordinate of bounding box origin in um
    xcoord_um: X coordinate in um
    ycoord_um: Y coordinate in um
    
    Returns
    -----------
    xcoord_rel_min_px: X coordinate in px with origin on bounding box
    ycoord_rel_min_px: Y coordinate in px with origin on bounding box
    """
    
    # convert to relative coords
    xcoord_rel_min, ycoord_rel_min =  change_origin(bbox_min_x_um, 
                         bbox_min_y_um,                          
                         xcoord_um,
                         ycoord_um)
    
    # convert um to px
    xcoord_rel_min_px, ycoord_rel_min_px = convert_to_px_coords(microns_per_pixel, 
                         xcoord_rel_min, 
                         ycoord_rel_min)
    
    return [xcoord_rel_min_px, ycoord_rel_min_px]

In [7]:
segmentation_stats = pd.read_csv(stats_path)
segmentation_stats = segmentation_stats.set_index("cell")
counts = pd.read_csv(counts_path)
counts = counts.set_index("cell")
segmentation = pd.read_csv(segmentation_path,
                          dtype = {'gene_reserved': str})

In [11]:
segmentation_plot = segmentation.loc[segmentation['is_noise'] == False]

segmentation_stats = segmentation_stats[segmentation_stats['elongation'].notnull()] 


In [12]:
counts_filt = counts.loc[counts.index.isin(segmentation_stats.index)]
adata = sc.AnnData(counts_filt, obsm={"spatial": np.array(segmentation_stats[["x", "y"]])})
adata.var_names = counts_filt.columns

In [22]:
adata.obs['n_transcripts'] = segmentation_stats['n_transcripts'].values
adata.obs['density'] = segmentation_stats['density'].values
adata.obs['elongation'] = segmentation_stats['elongation'].values
adata.obs['area'] = segmentation_stats['area'].values
adata.obs['avg_confidence'] = segmentation_stats['avg_confidence'].values
adata.obs['cluster'] = segmentation_stats['cluster'].values
adata.obs['cluster'] = adata.obs['cluster'].astype('string')
adata.obs['x_um'] = segmentation_stats['x'].values
adata.obs['y_um'] = segmentation_stats['y'].values
adata.obsm["spatial"] = adata.obsm["spatial"]*10


In [14]:
# normalize counts per cell to the median value of counts per cell
sc.pp.normalize_per_cell(adata)
# log norm
sc.pp.log1p(adata)
# determine the highly variable genes to use 
sc.pp.highly_variable_genes(adata)
# scale data 
sc.pp.scale(adata)
# pca
sc.tl.pca(adata)
sc.pp.neighbors(adata, n_neighbors = n_neighbors, n_pcs = n_pcs)
sc.tl.umap(adata)

In [15]:
adata2 = sc.AnnData(segmentation_stats[["n_transcripts", "density", "elongation", "area", "avg_confidence"]], obsm={"spatial": (np.array(segmentation_stats[["x", "y"]])),
                                             "X_umap" : adata.obsm["X_umap"]})
adata2.var_names = ["n_transcripts", "density", "elongation", "area", "avg_confidence"]
sc.pp.scale(adata2)

adata2.obsm["spatial"] = adata.obsm["spatial"]*10
adata2.obs['n_transcripts'] = segmentation_stats['n_transcripts'].values
adata2.obs['density'] = segmentation_stats['density'].values
adata2.obs['elongation'] = segmentation_stats['elongation'].values
adata2.obs['area'] = segmentation_stats['area'].values
adata2.obs['avg_confidence'] = segmentation_stats['avg_confidence'].values
adata2.obs['cluster'] = segmentation_stats['cluster'].values
adata2.obs['cluster'] = adata.obs['cluster'].astype('string')
#adata.obs['cluster'] = adata.obs['cluster'].astype('category')
adata2.obs['x_um'] = segmentation_stats['x'].values
adata2.obs['y_um'] = segmentation_stats['y'].values

/opt/conda/lib/python3.10/site-packages/anndata/utils.py:141: UserWarning: X converted to numpy array with dtype float64
  warnings.warn(f"{name} converted to numpy array with dtype {arr.dtype}")


In [21]:
vc = VitessceConfig(name='MERFISH', description='')
dataset = vc.add_dataset(name='Transcripts').add_object(AnnDataWrapper(
        adata2,
        mappings_obsm=["X_umap"],
        mappings_obsm_names=["UMAP",],
        cell_set_obs=["cluster"],
        cell_set_obs_names=["cluster"],
        spatial_centroid_obsm="spatial",
        expression_matrix="X"    )
                                                 )

scatterplot = vc.add_view(cm.SCATTERPLOT, dataset=dataset, mapping="UMAP")
cell_sets = vc.add_view(cm.CELL_SETS, dataset=dataset)
heatmap = vc.add_view(cm.HEATMAP, dataset=dataset)
genes = vc.add_view(cm.GENES, dataset=dataset)
spatial = vc.add_view(cm.SPATIAL, dataset=dataset)
set_exp = vc.add_view(cm.CELL_SET_EXPRESSION, dataset = dataset)

vc.layout((scatterplot |cell_sets)/ (genes | set_exp) / (heatmap |spatial));
vw = vc.widget(proxy = True,  theme = "light", height = 1000)
display(vw)

[2023-02-15 19:13:25 +0000] [57036] [INFO] Running on http://127.0.0.1:8005 (CTRL + C to quit)


VitessceWidget(config={'version': '1.0.7', 'name': 'MERFISH', 'description': '', 'datasets': [{'uid': 'A', 'na…

In [23]:
vc = VitessceConfig(name='MERFISH', description='')
dataset = vc.add_dataset(name='Transcripts').add_object(AnnDataWrapper(
        adata,
        mappings_obsm=["X_umap"],
        mappings_obsm_names=["UMAP",],
        cell_set_obs=["cluster"],
        cell_set_obs_names=["cluster"],
        spatial_centroid_obsm="spatial",
        expression_matrix="X"    )
                                                 )

scatterplot = vc.add_view(cm.SCATTERPLOT, dataset=dataset, mapping="UMAP")
cell_sets = vc.add_view(cm.CELL_SETS, dataset=dataset)
heatmap = vc.add_view(cm.HEATMAP, dataset=dataset)
genes = vc.add_view(cm.GENES, dataset=dataset)
spatial = vc.add_view(cm.SPATIAL, dataset=dataset)
set_exp = vc.add_view(cm.CELL_SET_EXPRESSION, dataset = dataset)

vc.layout((scatterplot |cell_sets)/ (genes | set_exp) / (heatmap |spatial));
vw = vc.widget(proxy = True,  theme = "light", height = 1000)
display(vw)

/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'cluster' as categorical
[2023-02-15 19:14:12 +0000] [57036] [INFO] Running on http://127.0.0.1:8006 (CTRL + C to quit)


VitessceWidget(config={'version': '1.0.7', 'name': 'MERFISH', 'description': '', 'datasets': [{'uid': 'A', 'na…